In [6]:
from datetime import datetime, timedelta
from typing import List
import math
import faiss
import os
import logging
logging.basicConfig(level=logging.ERROR)
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain.vectorstores import FAISS
from termcolor import colored
from langchain_experimental.generative_agents import (
    GenerativeAgent,
    GenerativeAgentMemory,
)

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY', '')

In [7]:
USER_NAME = "Heena"  # The name you want to use when interviewing the agent.

LLM = ChatOpenAI(max_tokens=1500)  # Can be any LLM you want.

### Implementing your first Generative Agent

In [8]:
def relevance_score_fn(score: float) -> float:
    """Return a similarity score on a scale [0, 1]."""
    # This will differ depending on a few things:
    # - the distance / similarity metric used by the VectorStore
    # - the scale of your embeddings (OpenAI's are unit norm. Many others are not!)
    # This function converts the euclidean norm of normalized embeddings
    # (0 is most similar, sqrt(2) most dissimilar)
    # to a similarity function (0 to 1)
    return 1.0 - score / math.sqrt(2)


def create_new_memory_retriever():
    """Create a new vector store retriever unique to the agent."""
    # Define your embedding model
    embeddings_model = OpenAIEmbeddings()
    # Initialize the vectorstore as empty
    embedding_size = 1536
    index = faiss.IndexFlatL2(embedding_size)
    vectorstore = FAISS(
        embeddings_model.embed_query,
        index,
        InMemoryDocstore({}),
        {},
        relevance_score_fn=relevance_score_fn,
    )
    return TimeWeightedVectorStoreRetriever(
        vectorstore=vectorstore, other_score_keys=["importance"], k=15
    )

In [9]:
alexis_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=8,  # we will give this a relatively low number to show how reflection works
)

# Defining the Generative Agent: Alexis
alexis = GenerativeAgent(
    name="Alexis",
    age=30,
    traits="curious, creative writer, world traveler",  # Persistent traits of Alexis
    status="exploring the intersection of technology and storytelling",  # Current status of Alexis
    memory_retriever=create_new_memory_retriever(),
    llm=LLM,
    memory=alexis_memory,
)

c:\Users\heena\Desktop\Genetative_AI\genai_env\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [10]:
print(alexis.get_summary())

Name: Alexis (age: 30)
Innate traits: curious, creative writer, world traveler
Alexis is a hardworking, determined, and detail-oriented individual who is constantly seeking knowledge and growth in her career. She is passionate and committed to her work, often going above and beyond to achieve her goals. Alexis is also organized and focused, with a strong ability to problem-solve and adapt to different situations.


In [11]:
# We can add memories directly to the memory object

alexis_observations = [
    "Alexis recalls her morning walk in the park",
    "Alexis feels excited about the new book she started reading",
    "Alexis remembers her conversation with a close friend",
    "Alexis thinks about the painting she saw at the art gallery",
    "Alexis is planning to learn a new recipe for dinner",
    "Alexis is looking forward to her weekend trip",
    "Alexis contemplates her goals for the month."
]

for observation in alexis_observations:
    alexis.memory.add_memory(observation)



# We will see how this summary updates after more observations to create a more rich description.
print(alexis.get_summary(force_refresh=True))

Name: Alexis (age: 30)
Innate traits: curious, creative writer, world traveler
Alexis is thoughtful, goal-oriented, enjoys socializing with friends, appreciates nature and art, likes reading, and is eager to try new things such as learning a new recipe.


### Interacting and Providing Context to Generative Characters

### Pre-Interview with Character
Before sending our character on their way, let's ask them a few questions.


In [12]:
def interview_agent(agent: GenerativeAgent, message: str) -> str:
    """Help the notebook user interact with the agent."""
    new_message = f"{USER_NAME} says {message}"
    return agent.generate_dialogue_response(new_message)[1]

In [13]:
interview_agent(alexis, "What do you like to do?")

'Alexis said "I enjoy writing, traveling, spending time with friends, exploring nature, and trying new things like learning new recipes. How about you, Heena?"'

### Step through the day's observations.

In [14]:
# Let's give Alexis a series of observations to reflect on her day
# Adding observations to Alexis' memory
alexis_observations_day = [
    "Alexis starts her day with a refreshing yoga session.",
    "Alexis spends time writing in her journal.",
    "Alexis experiments with a new recipe she found online.",
    "Alexis gets lost in her thoughts while gardening.",
    "Alexis decides to call her grandmother for a heartfelt chat.",
    "Alexis relaxes in the evening by playing her favorite piano pieces.",
]

for observation in alexis_observations_day:
    alexis.memory.add_memory(observation)

In [15]:
# Let's observe how Alexis's day influences her memory and character
for i, observation in enumerate(alexis_observations_day):
    _, reaction = alexis.generate_reaction(observation)
    print(colored(observation, "green"), reaction)
    if ((i + 1) % len(alexis_observations_day)) == 0:
        print("*" * 40)
        print(
            colored(
                f"After these observations, Alexis's summary is:\n{alexis.get_summary(force_refresh=True)}",
                "blue",
            )
        )
        print("*" * 40)

Alexis starts her day with a refreshing yoga session. Alexis feels energized and ready to start her day.
Alexis spends time writing in her journal. Alexis finds solace and clarity in journaling.
Alexis experiments with a new recipe she found online. Alexis enjoys trying out new recipes and finds joy in experimenting in the kitchen.
Alexis gets lost in her thoughts while gardening. Alexis finds peace and connection with nature while getting lost in her thoughts while gardening.
Alexis decides to call her grandmother for a heartfelt chat. Alexis feels grateful for the opportunity to connect with her grandmother.
Alexis relaxes in the evening by playing her favorite piano pieces. Alexis finds comfort and relaxation in playing her favorite piano pieces.
****************************************
After these observations, Alexis's summary is:
Name: Alexis (age: 30)
Innate traits: curious, creative writer, world traveler
Alexis is a reflective and introspective individual who finds solace in j